<a href="https://colab.research.google.com/github/rajy4683/EVAP2/blob/master/ENDS9_Attn_AmbigQa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Jan  7 17:29:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%bash
python -m spacy download en
python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907057 sha256=1f6380b0c7b2679cbea0b13a29d5a06a28019a7bdd460494d8c25855b027b404
  Stored in directory: /tmp/pip-ephem-wheel-cache-5sojlhy5/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


**Restart Notebook**

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np
import pandas as pd
import random
import math
import time
import json

Set the random seeds for reproducability.

In [4]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Load the German and English spaCy models.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [7]:
!unzip /content/ambignq_light.zip

Archive:  /content/ambignq_light.zip
   creating: ambignq_light/
  inflating: ambignq_light/dev_light.json  
  inflating: ambignq_light/LICENSE   
  inflating: ambignq_light/train_light.json  
  inflating: dev_light.json          
  inflating: LICENSE                 
  inflating: train_light.json        


In [ ]:
sentence = []
label = []
with open('/content/OpenBookQa/train.jsonl') as h:
    for line in h:
        example = json.loads(line)
        print(example.keys())
        # scores = []
        merged_choices = ' A: '.join([choice['text'] for choice in example['question']['choices']])
        input = 'Q: ' + example['question']['stem'] + ' A: ' + merged_choices
        correct_answer = [ choice['text'] for choice in example['question']['choices'] if choice['label'] == example['answerKey'] ][0]
        sentence.append(input)
        label.append(correct_answer)
        #print(input, correct_answer)
dataset_df = pd.DataFrame({'sentence':sentence, 'label':label})

In [ ]:
dataset_df

,sentence,label
0,Q: The sun is responsible for A: puppies learn...,"plants sprouting, blooming and wilting"
1,Q: When standing miles away from Mount Rushmor...,the mountains seem smaller than in photographs
2,Q: When food is reduced in the stomach A: the ...,nutrients are being deconstructed
3,Q: Stars are A: warm lights that float A: made...,great balls of gas burning billions of miles away
4,Q: You can make a telescope with a A: straw A:...,mailing tube
...,...,...
4952,Q: A bulldozer alters the area of A: skyscrape...,skyscrapers
4953,Q: An organism that can survive without the he...,Brewer's yeast
4954,Q: The nimbleness of this animal is a key adap...,the antelope
4955,Q: Birds will have different kinds of beaks de...,organisms they hunt


In [10]:
import itertools
import json
import os
import csv
import errno
import random
from random import shuffle
from typing import List
import spacy
from tqdm import tqdm

In [8]:
def ambigqa():
    def read_file(file, dir, split):
        outfile = open(f"{dir}/{split}.tsv", "w")
        outfile_meta = open(f"{dir}/{split}_meta.tsv", "w")
        size = 0
        with open(file, "r") as f:
            json_file = json.load(f)
            for item in tqdm(json_file):
                question = item['question'].replace("\n", " ").replace("\t", " ")
                single_answers_already_included = []
                for anno in item["annotations"]:
                    if anno['type'] == "singleAnswer":
                        for ans in anno['answer']:
                            if ans not in single_answers_already_included:
                                ans = ans.replace("\n", " ").replace("\t", " ")
                                outfile.write(f"{question}\t{ans}\n")
                                outfile_meta.write(item['id'] + "\n")
                                single_answers_already_included.append(ans)
                                size += 1
                    else:
                        answers = []
                        for x in anno['qaPairs']:
                            answers.append(x['answer'][0])

                        answers = [x.strip() for x in answers]
                        answers = list(set(answers))  # to drop duplicate answers
                        for i, ordering in enumerate(itertools.permutations(answers)):
                            if i >= 3:
                                break
                            ans_str = " [SEP] ".join(ordering).replace("\n", " ").replace("\t", " ")
                            outfile.write(f"{question}\t{ans_str}\n")
                            outfile_meta.write(item['id'] + "\n")
                            size += 1
        return size

    count_dev = read_file("/content/ambignq_light/dev_light.json", "/content/ambigqa", "dev")
    count_train = read_file("/content/ambignq_light/train_light.json", "/content/ambigqa", "train")
    count_test = 0

    # Create TSVs and get counts.
    with open("ambigqa/counts.json", "w") as outfile:
        json.dump({"train": count_train, "dev": count_dev, "test": count_test}, outfile)

In [11]:
ambigqa()

100%|██████████| 10036/10036 [00:00<00:00, 243129.80it/s]


In [12]:
#dataset = pd.read_csv("/content/dev.tsv", sep='\t', header=None)
dataset = pd.read_csv("/content/ambigqa/train.tsv", sep='\t', header=None)
dataset.columns = ["sentence", "label"]

We create the tokenizers.

In [13]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

The fields remain the same as before.

Load the data.

In [14]:
from torchtext import data 
import matplotlib.pyplot as plt
Sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =False, include_lengths=False, lower=False)
Label = data.Field(sequential =True, tokenize ='spacy', is_target=False, batch_first =False, include_lengths=False, lower=False)

## Pruned dataset
Dataset was pruned to 5000 entries given the size of the dataset

In [15]:
#dataset = dataset_df
fields = [('sentence', Sentence),('label',Label)]
#example = [data.Example.fromlist([dataset.sentence[i],dataset.label[i]], fields) for i in range(dataset.shape[0])] 
example = [data.Example.fromlist([dataset.sentence[i],dataset.label[i]], fields) for i in range(5000)] 
commonqa_ds = data.Dataset(example, fields)
(train, valid) = commonqa_ds.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [16]:
Sentence.build_vocab(train)
Label.build_vocab(train)

In [17]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  4450
Size of label vocab :  5708
Top 10 words appreared repeatedly : [('?', 4250), ('the', 3719), ('Who', 1797), ('in', 1581), ('of', 1437), ('is', 899), ('When', 842), ('did', 578), ('What', 540), ('was', 521)]
Labels :  defaultdict(<function _default_unk_index at 0x7f7460c55a60>, {'<unk>': 0, '<pad>': 1, 'SEP': 2, '[': 3, ']': 4, ',': 5, 'and': 6, 'the': 7, '2017': 8, 'of': 9, '-': 10, '2018': 11, 'September': 12, 'July': 13, 'November': 14, 'October': 15, 'May': 16, 'The': 17, 'August': 18, 'March': 19, 'April': 20, 'June': 21, '"': 22, '1': 23, '2016': 24, 'December': 25, 'January': 26, '2015': 27, '12': 28, '15': 29, 'John': 30, '2': 31, '6': 32, '25': 33, '13': 34, '16': 35, 'Michael': 36, 'in': 37, '17': 38, '23': 39, '20': 40, 'New': 41, ')': 42, '21': 43, '10': 44, '9': 45, 'February': 46, '(': 47, 'United': 48, '3': 49, '7': 50, '8': 51, 'James': 52, '26': 53, '18': 54, '5': 55, 'to': 56, "'s": 57, '11': 58, '14': 59, '30': 60, 'a': 61, '2012': 62, '4':

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
BATCH_SIZE=16
train_iterator, test_iterator = BucketIterator.splits(
    (train, valid), 
    batch_size = BATCH_SIZE, 
    device = device, sort=False, shuffle=False)

Build the vocabulary.

Define the device.

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Create the iterators.

## Building the Seq2Seq Model

### Encoder

First, we'll build the encoder. Similar to the previous model, we only use a single layer GRU, however we now use a *bidirectional RNN*. With a bidirectional RNN, we have two RNNs in each layer. A *forward RNN* going over the embedded sentence from left to right (shown below in green), and a *backward RNN* going over the embedded sentence from right to left (teal). All we need to do in code is set `bidirectional = True` and then pass the embedded sentence to the RNN as before. 

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq8.png?raw=1)

We now have:

$$\begin{align*}
h_t^\rightarrow &= \text{EncoderGRU}^\rightarrow(e(x_t^\rightarrow),h_{t-1}^\rightarrow)\\
h_t^\leftarrow &= \text{EncoderGRU}^\leftarrow(e(x_t^\leftarrow),h_{t-1}^\leftarrow)
\end{align*}$$

Where $x_0^\rightarrow = \text{<sos>}, x_1^\rightarrow = \text{guten}$ and $x_0^\leftarrow = \text{<eos>}, x_1^\leftarrow = \text{morgen}$.

As before, we only pass an input (`embedded`) to the RNN, which tells PyTorch to initialize both the forward and backward initial hidden states ($h_0^\rightarrow$ and $h_0^\leftarrow$, respectively) to a tensor of all zeros. We'll also get two context vectors, one from the forward RNN after it has seen the final word in the sentence, $z^\rightarrow=h_T^\rightarrow$, and one from the backward RNN after it has seen the first word in the sentence, $z^\leftarrow=h_T^\leftarrow$.

The RNN returns `outputs` and `hidden`. 

`outputs` is of size **[src len, batch size, hid dim * num directions]** where the first `hid_dim` elements in the third axis are the hidden states from the top layer forward RNN, and the last `hid_dim` elements are hidden states from the top layer backward RNN. We can think of the third axis as being the forward and backward hidden states concatenated together other, i.e. $h_1 = [h_1^\rightarrow; h_{T}^\leftarrow]$, $h_2 = [h_2^\rightarrow; h_{T-1}^\leftarrow]$ and we can denote all encoder hidden states (forward and backwards concatenated together) as $H=\{ h_1, h_2, ..., h_T\}$.

`hidden` is of size **[n layers * num directions, batch size, hid dim]**, where **[-2, :, :]** gives the top layer forward RNN hidden state after the final time-step (i.e. after it has seen the last word in the sentence) and **[-1, :, :]** gives the top layer backward RNN hidden state after the final time-step (i.e. after it has seen the first word in the sentence).

As the decoder is not bidirectional, it only needs a single context vector, $z$, to use as its initial hidden state, $s_0$, and we currently have two, a forward and a backward one ($z^\rightarrow=h_T^\rightarrow$ and $z^\leftarrow=h_T^\leftarrow$, respectively). We solve this by concatenating the two context vectors together, passing them through a linear layer, $g$, and applying the $\tanh$ activation function. 

$$z=\tanh(g(h_T^\rightarrow, h_T^\leftarrow)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$$

**Note**: this is actually a deviation from the paper. Instead, they feed only the first backward RNN hidden state through a linear layer to get the context vector/decoder initial hidden state. This doesn't seem to make sense to me, so we have changed it.

As we want our model to look back over the whole of the source sentence we return `outputs`, the stacked forward and backward hidden states for every token in the source sentence. We also return `hidden`, which acts as our initial hidden state in the decoder.

In [21]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

### Attention

Next up is the attention layer. This will take in the previous hidden state of the decoder, $s_{t-1}$, and all of the stacked forward and backward hidden states from the encoder, $H$. The layer will output an attention vector, $a_t$, that is the length of the source sentence, each element is between 0 and 1 and the entire vector sums to 1.

Intuitively, this layer takes what we have decoded so far, $s_{t-1}$, and all of what we have encoded, $H$, to produce a vector, $a_t$, that represents which words in the source sentence we should pay the most attention to in order to correctly predict the next word to decode, $\hat{y}_{t+1}$. 

First, we calculate the *energy* between the previous decoder hidden state and the encoder hidden states. As our encoder hidden states are a sequence of $T$ tensors, and our previous decoder hidden state is a single tensor, the first thing we do is `repeat` the previous decoder hidden state $T$ times. We then calculate the energy, $E_t$, between them by concatenating them together and passing them through a linear layer (`attn`) and a $\tanh$ activation function. 

$$E_t = \tanh(\text{attn}(s_{t-1}, H))$$ 

This can be thought of as calculating how well each encoder hidden state "matches" the previous decoder hidden state.

We currently have a **[dec hid dim, src len]** tensor for each example in the batch. We want this to be **[src len]** for each example in the batch as the attention should be over the length of the source sentence. This is achieved by multiplying the `energy` by a **[1, dec hid dim]** tensor, $v$.

$$\hat{a}_t = v E_t$$

We can think of $v$ as the weights for a weighted sum of the energy across all encoder hidden states. These weights tell us how much we should attend to each token in the source sequence. The parameters of $v$ are initialized randomly, but learned with the rest of the model via backpropagation. Note how $v$ is not dependent on time, and the same $v$ is used for each time-step of the decoding. We implement $v$ as a linear layer without a bias.

Finally, we ensure the attention vector fits the constraints of having all elements between 0 and 1 and the vector summing to 1 by passing it through a $\text{softmax}$ layer.

$$a_t = \text{softmax}(\hat{a_t})$$

This gives us the attention over the source sentence!

Graphically, this looks something like below. This is for calculating the very first attention vector, where $s_{t-1} = s_0 = z$. The green/teal blocks represent the hidden states from both the forward and backward RNNs, and the attention computation is all done within the pink block.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq9.png?raw=1)

In [22]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

### Decoder

Next up is the decoder. 

The decoder contains the attention layer, `attention`, which takes the previous hidden state, $s_{t-1}$, all of the encoder hidden states, $H$, and returns the attention vector, $a_t$.

We then use this attention vector to create a weighted source vector, $w_t$, denoted by `weighted`, which is a weighted sum of the encoder hidden states, $H$, using $a_t$ as the weights.

$$w_t = a_t H$$

The embedded input word, $d(y_t)$, the weighted source vector, $w_t$, and the previous decoder hidden state, $s_{t-1}$, are then all passed into the decoder RNN, with $d(y_t)$ and $w_t$ being concatenated together.

$$s_t = \text{DecoderGRU}(d(y_t), w_t, s_{t-1})$$

We then pass $d(y_t)$, $w_t$ and $s_t$ through the linear layer, $f$, to make a prediction of the next word in the target sentence, $\hat{y}_{t+1}$. This is done by concatenating them all together.

$$\hat{y}_{t+1} = f(d(y_t), w_t, s_t)$$

The image below shows decoding the first word in an example translation.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq10.png?raw=1)

The green/teal blocks show the forward/backward encoder RNNs which output $H$, the red block shows the context vector, $z = h_T = \tanh(g(h^\rightarrow_T,h^\leftarrow_T)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$, the blue block shows the decoder RNN which outputs $s_t$, the purple block shows the linear layer, $f$, which outputs $\hat{y}_{t+1}$ and the orange block shows the calculation of the weighted sum over $H$ by $a_t$ and outputs $w_t$. Not shown is the calculation of $a_t$.

In [23]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

### Seq2Seq

This is the first model where we don't have to have the encoder RNN and decoder RNN have the same hidden dimensions, however the encoder has to be bidirectional. This requirement can be removed by changing all occurences of `enc_dim * 2` to `enc_dim * 2 if encoder_is_bidirectional else enc_dim`. 

This seq2seq encapsulator is similar to the last two. The only difference is that the `encoder` returns both the final hidden state (which is the final hidden state from both the forward and backward encoder RNNs passed through a linear layer) to be used as the initial hidden state for the decoder, as well as every hidden state (which are the forward and backward hidden states stacked on top of each other). We also need to ensure that `hidden` and `encoder_outputs` are passed to the decoder. 

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive $z$ and $H$
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and all encoder outputs, $H$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate

In [24]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

## Training the Seq2Seq Model

The rest of this session is very similar to the previous one.

We initialise our parameters, encoder, decoder and seq2seq model (placing it on the GPU if we have one). 

In [25]:
# INPUT_DIM = len(SRC.vocab)
# OUTPUT_DIM = len(TRG.vocab)

INPUT_DIM = len(Sentence.vocab)
OUTPUT_DIM = len(Label.vocab)

ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

We use a simplified version of the weight initialization scheme used in the paper. Here, we will initialize all biases to zero and all weights from $\mathcal{N}(0, 0.01)$.

In [26]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(4450, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5708, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5708, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Calculate the number of parameters. We get an increase of almost 50% in the amount of parameters from the last model. 

In [27]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 19,268,172 trainable parameters


We create an optimizer.

In [33]:
optimizer = optim.Adam(model.parameters())

We initialize the loss function.

In [28]:
# TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

TRG_PAD_IDX = Label.vocab.stoi[Label.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [29]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        # src = batch.src
        # trg = batch.trg
        src = batch.sentence
        trg = batch.label
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [30]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            # src = batch.src
            # trg = batch.trg
            src = batch.sentence
            trg = batch.label

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Finally, define a timing function.

In [31]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [34]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 31s
	Train Loss: 5.466 | Train PPL: 236.482
	 Val. Loss: 5.712 |  Val. PPL: 302.455
Epoch: 02 | Time: 0m 31s
	Train Loss: 4.120 | Train PPL:  61.588
	 Val. Loss: 5.679 |  Val. PPL: 292.736
Epoch: 03 | Time: 0m 32s
	Train Loss: 3.579 | Train PPL:  35.840
	 Val. Loss: 5.619 |  Val. PPL: 275.518
Epoch: 04 | Time: 0m 32s
	Train Loss: 3.051 | Train PPL:  21.138
	 Val. Loss: 5.485 |  Val. PPL: 240.979
Epoch: 05 | Time: 0m 32s
	Train Loss: 2.512 | Train PPL:  12.323
	 Val. Loss: 5.458 |  Val. PPL: 234.530
Epoch: 06 | Time: 0m 32s
	Train Loss: 2.004 | Train PPL:   7.421
	 Val. Loss: 5.292 |  Val. PPL: 198.768
Epoch: 07 | Time: 0m 32s
	Train Loss: 1.661 | Train PPL:   5.264
	 Val. Loss: 5.156 |  Val. PPL: 173.396
Epoch: 08 | Time: 0m 32s
	Train Loss: 1.414 | Train PPL:   4.113
	 Val. Loss: 5.293 |  Val. PPL: 198.993
Epoch: 09 | Time: 0m 32s
	Train Loss: 1.238 | Train PPL:   3.450
	 Val. Loss: 5.189 |  Val. PPL: 179.292
Epoch: 10 | Time: 0m 32s
	Train Loss: 1.090 | Train PPL

In [35]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [39]:
N_EPOCHS = 30
CLIP = 1
model.load_state_dict(torch.load('tut3-model.pt'))
best_valid_loss = 4.9866932503720545

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 32s
	Train Loss: 1.011 | Train PPL:   2.749
	 Val. Loss: 4.860 |  Val. PPL: 128.971
Epoch: 02 | Time: 0m 32s
	Train Loss: 0.898 | Train PPL:   2.455
	 Val. Loss: 4.669 |  Val. PPL: 106.548
Epoch: 03 | Time: 0m 32s
	Train Loss: 0.820 | Train PPL:   2.270
	 Val. Loss: 4.567 |  Val. PPL:  96.222
Epoch: 04 | Time: 0m 32s
	Train Loss: 0.781 | Train PPL:   2.183
	 Val. Loss: 4.513 |  Val. PPL:  91.229
Epoch: 05 | Time: 0m 32s
	Train Loss: 0.727 | Train PPL:   2.069
	 Val. Loss: 4.488 |  Val. PPL:  88.941
Epoch: 06 | Time: 0m 32s
	Train Loss: 0.693 | Train PPL:   1.999
	 Val. Loss: 4.436 |  Val. PPL:  84.466
Epoch: 07 | Time: 0m 32s
	Train Loss: 0.668 | Train PPL:   1.949
	 Val. Loss: 4.415 |  Val. PPL:  82.642
Epoch: 08 | Time: 0m 32s
	Train Loss: 0.645 | Train PPL:   1.907
	 Val. Loss: 4.404 |  Val. PPL:  81.779
Epoch: 09 | Time: 0m 32s
	Train Loss: 0.617 | Train PPL:   1.853
	 Val. Loss: 4.371 |  Val. PPL:  79.144
Epoch: 10 | Time: 0m 32s
	Train Loss: 0.584 | Train PPL

In [38]:
optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.0001)

In [41]:
best_valid_loss, optimizer

(4.023710258463596, Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 1e-05
     weight_decay: 0.0001
 ))

In [42]:
#optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.01)
N_EPOCHS = 30
CLIP = 1
model.load_state_dict(torch.load('tut3-model.pt'))
best_valid_loss = 4.023710258463596

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 32s
	Train Loss: 0.406 | Train PPL:   1.501
	 Val. Loss: 3.998 |  Val. PPL:  54.497
Epoch: 02 | Time: 0m 32s
	Train Loss: 0.386 | Train PPL:   1.471
	 Val. Loss: 3.994 |  Val. PPL:  54.276
Epoch: 03 | Time: 0m 32s
	Train Loss: 0.377 | Train PPL:   1.459
	 Val. Loss: 3.982 |  Val. PPL:  53.641
Epoch: 04 | Time: 0m 32s
	Train Loss: 0.385 | Train PPL:   1.469
	 Val. Loss: 3.959 |  Val. PPL:  52.389
Epoch: 05 | Time: 0m 32s
	Train Loss: 0.390 | Train PPL:   1.476
	 Val. Loss: 3.957 |  Val. PPL:  52.317
Epoch: 06 | Time: 0m 32s
	Train Loss: 0.380 | Train PPL:   1.462
	 Val. Loss: 3.940 |  Val. PPL:  51.425
Epoch: 07 | Time: 0m 32s
	Train Loss: 0.371 | Train PPL:   1.449
	 Val. Loss: 3.917 |  Val. PPL:  50.255
Epoch: 08 | Time: 0m 32s
	Train Loss: 0.362 | Train PPL:   1.436
	 Val. Loss: 3.924 |  Val. PPL:  50.610
Epoch: 09 | Time: 0m 32s
	Train Loss: 0.362 | Train PPL:   1.437
	 Val. Loss: 3.899 |  Val. PPL:  49.376
Epoch: 10 | Time: 0m 32s
	Train Loss: 0.364 | Train PPL

In [44]:
best_valid_loss, optimizer

(3.760860301078634, Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 1e-05
     weight_decay: 0.0001
 ))

In [45]:
optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.001)

In [46]:
N_EPOCHS = 50
CLIP = 1
model.load_state_dict(torch.load('tut3-model.pt'))
best_valid_loss = 3.760860301078634

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)

    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 32s
	Train Loss: 0.329 | Train PPL:   1.390
	 Val. Loss: 3.761 |  Val. PPL:  42.982
Epoch: 02 | Time: 0m 32s
	Train Loss: 0.353 | Train PPL:   1.423
	 Val. Loss: 3.742 |  Val. PPL:  42.177
Epoch: 03 | Time: 0m 32s
	Train Loss: 0.368 | Train PPL:   1.444
	 Val. Loss: 3.751 |  Val. PPL:  42.558
Epoch: 04 | Time: 0m 32s
	Train Loss: 0.397 | Train PPL:   1.487
	 Val. Loss: 3.703 |  Val. PPL:  40.586
Epoch: 05 | Time: 0m 32s
	Train Loss: 0.418 | Train PPL:   1.519
	 Val. Loss: 3.705 |  Val. PPL:  40.653
Epoch: 06 | Time: 0m 32s
	Train Loss: 0.438 | Train PPL:   1.550
	 Val. Loss: 3.686 |  Val. PPL:  39.873
Epoch: 07 | Time: 0m 32s
	Train Loss: 0.449 | Train PPL:   1.566
	 Val. Loss: 3.658 |  Val. PPL:  38.781
Epoch: 08 | Time: 0m 32s
	Train Loss: 0.471 | Train PPL:   1.601
	 Val. Loss: 3.673 |  Val. PPL:  39.355
Epoch: 09 | Time: 0m 32s
	Train Loss: 0.498 | Train PPL:   1.645
	 Val. Loss: 3.665 |  Val. PPL:  39.060
Epoch: 10 | Time: 0m 32s
	Train Loss: 0.510 | Train PPL

In [ ]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 1e-05
    weight_decay: 0.01
)

In [47]:
best_valid_loss

3.656712980980569

Finally, we test the model on the test set using these "best" parameters.

In [48]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.657 | Test PPL:  38.734 |
